In [ ]:
# flag = 0

# if flag == 0:
#   !pip install pandas numpy scikit-learn seaborn matplotlib scipy nltk tensorflow keras transformers
#   flag = 1

import tensorflow as tf
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import concatenate, Concatenate
from keras.layers import Input, Embedding, Conv1D, Conv2D, GlobalMaxPooling1D, GlobalAveragePooling1D, Flatten, MaxPooling2D, MaxPooling1D, Dense, Dropout, Reshape
from keras.models import Model
from sklearn.metrics import accuracy_score
import transformers
import requests
import zipfile
import io
import os
import glob
import re
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers.legacy import Adam
from sklearn.metrics import mean_squared_error
import time
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from keras.models import load_model
from scipy.stats import ks_2samp

#### Read the dataset

In [ ]:
df = pd.read_csv('SDSS_DR18.csv')
df.head()

#### Check column types

In [ ]:
df.info()

#### Check duplicates

In [ ]:
num_duplicate_rows = df.duplicated().sum()

print("Number of duplicate rows:", num_duplicate_rows)

#### Check for missing values

In [ ]:
column_names = df.columns.values.tolist()

print("Column name \t Count of missing values \t Percentage of missing value to total rows")
for col in column_names:
    count_nan = df[col].isnull().sum()
    pct_nan = count_nan / len(df) * 100
    if col in ['ra', 'dec', 'u', 'g', 'r', 'i', 'z']:
        print(col + " - \t\t\t" + str(count_nan) + " \t\t\t\t" + str(round(pct_nan, 2)) + "%")
    else:
        print(col + " - \t\t" + str(count_nan) + " \t\t\t\t" + str(round(pct_nan, 2)) + "%")

#### Bar representing instances per class

In [ ]:
class_counts = df['class'].value_counts()

plt.bar(class_counts.index, class_counts.values)
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.show()

#### Box plots representing the statistical summary of all columns

In [ ]:
columns_to_exclude = ['objid', 'specobjid', 'class']

column_names = [col for col in df.columns if col not in columns_to_exclude]

num_columns = len(column_names)
num_rows = math.ceil(num_columns / 5)

fig, axs = plt.subplots(num_rows, 5, figsize=(13, 3 * num_rows))

axs = axs.flatten()

for i, column in enumerate(column_names):
    current_row = i // 5
    position_in_row = i % 5

    ax = axs[i]

    sns.boxplot(x=df['class'], y=df[column], ax=ax)
    ax.set_title(column)
    ax.set_ylabel(column)

    ax.set_xlabel('')

    if current_row == 0 and position_in_row < 2:
        ax.set_title(column)
    else:
        current_row += 1

for i in range(num_columns, len(axs)):
    fig.delaxes(axs[i])

plt.tight_layout()
plt.show()


#### Histograms for representing data distribution across all columns

In [ ]:
num_columns = len(column_names)
num_rows = math.ceil(num_columns / 3)

plt.figure(figsize=(19, num_rows * 5))
plt.subplots_adjust(hspace=0.5)

for i, column in enumerate(column_names):
    plt.subplot(num_rows, 3, i + 1)
    plt.hist(df[column], bins=30, color='skyblue')
    plt.title(column)
    plt.xlabel('Values')
    plt.ylabel('Frequency')

plt.show()


#### Data split for model training

In [ ]:
X = df.drop(columns=['class', 'objid', 'specobjid'])
y = df['class']

In [ ]:
print(X.shape)
print(y.shape)

#### Shuffling the dataset

In [ ]:
X, y = shuffle(X, y, random_state=42)

#### Dataset split into train, test and validate

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42)

In [ ]:
num_classes = 3

#### Scaling all the numerical data to  standard scale

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

#### Label encoding the target feature

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

#### One hot encoding the target

In [ ]:
y_train_one_hot = to_categorical(y_train, num_classes)
y_val_one_hot = to_categorical(y_val, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)

#### Checking number of instances for train, test and validate

In [ ]:
X_train_rows = X_train.shape[0]
X_test_rows = X_test.shape[0]
X_val_rows = X_val.shape[0]
y_train_rows = y_train.shape[0]
y_test_rows = y_test.shape[0]
y_val_rows = y_val.shape[0]

print("Input for train:", X_train_rows)
print("Input for test:", X_test_rows)
print("Input for validation:", X_val_rows)
print("Target for train:", y_train_rows)
print("Target for test:", y_test_rows)
print("Target for validation:", y_val_rows)

#### Checking shape of input features

In [ ]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

#### Checking shape of target

In [ ]:
print("y_train shape:", y_train_one_hot.shape)
print("y_val shape:", y_val_one_hot.shape)
print("y_test shape:", y_test_one_hot.shape)

## Conditional GAN

In [ ]:
noise_dim = 100
batch_size = 32
epochs = 100
num_features = 33

In [ ]:
def build_conditional_generator(layers, num_classes):
    noise_input = Input(shape=(noise_dim,))
    class_input = Input(shape=(num_classes,))
    combined_input = Concatenate()([noise_input, class_input])
    model = Sequential()
    
    for units in layers:
        model.add(Dense(units, activation='relu', input_dim=noise_dim + num_classes))
        model.add(Dropout(0.5))
    model.add(Dense(X_train.shape[1], activation='linear'))
    generator_output = model(combined_input)
    generator = Model([noise_input, class_input], generator_output)

    return generator

In [ ]:
def build_conditional_discriminator(layers, num_classes):
    data_input = Input(shape=(num_features,))
    class_input = Input(shape=(num_classes,))
    combined_input = Concatenate()([data_input, class_input])

    model = Sequential()
    for units in layers:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    discriminator_output = model(combined_input)
    discriminator = Model([data_input, class_input], discriminator_output)

    return discriminator

In [ ]:
def build_cgan(generator, discriminator):
    discriminator.trainable = False
    gan_noise_input = Input(shape=(noise_dim,))
    class_input = Input(shape=(num_classes,))
    
    generated_data = generator([gan_noise_input, class_input])
    
    gan_output = discriminator([generated_data, class_input])
    
    gan = Model([gan_noise_input, class_input], gan_output)

    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

    return gan

In [ ]:
def monte_carlo_sampling(generator, X_val, class_labels, num_simulations=10):
    mse_list = []
    for _ in range(num_simulations):
        val_noise = np.random.normal(0, 1, size=[X_val.shape[0], noise_dim])
        generated_samples = generator.predict([val_noise, class_labels])
        mse = mean_squared_error(X_val, generated_samples)
        mse_list.append(mse)
    return np.mean(mse_list), np.std(mse_list)


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
final_mse = None
best_mse = float('inf')
best_weights = None
best_config = None
best_epoch = 0
patience = 5

In [ ]:
global_best_mse = float('inf')
global_best_config = None
global_best_weights = None

In [ ]:
layer_configs = [(32, 64), (64, 128), (64, 128, 256), (128, 256, 512)]

In [ ]:
mse_scores = []

In [ ]:
for layers in layer_configs:
    start_time = time.time()
    print(f"Start Time: {str(datetime.fromtimestamp(start_time))}")

    # Build cGAN
    generator = build_conditional_generator(layers, num_classes)
    discriminator = build_conditional_discriminator(layers, num_classes)
    gan = build_cgan(generator, discriminator)
    
#     gan.fit([X_train, y_train_one_hot], np.ones((len(X_train), 1)), validation_data=([X_val, y_val_one_hot], np.ones((len(X_val), 1))),
#             epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

    class_labels_for_validation = np.zeros((X_val.shape[0], num_classes))
    class_labels_for_validation[:, 2] = 1  

    best_mean_mse = float('inf')
    best_weights = None
    best_config = layers  

    # Training cGAN
    for epoch in range(epochs):
        for _ in range(X_train.shape[0] // batch_size):
            # Discriminator training
            noise = np.random.normal(0, 1, size=[batch_size, noise_dim])
            real_data = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]
            
            class_label = np.zeros((batch_size, num_classes))
            class_label[:, 2] = 1  
            
            combined_input = [noise, class_label]
            generated_sample = generator.predict(combined_input)
            
            combined_data = np.concatenate([real_data, generated_sample], axis=0)
            combined_class_labels = np.concatenate([class_label, class_label], axis=0)
            
            labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
            labels += 0.05 * np.random.random(labels.shape)           
            
            d_loss = discriminator.train_on_batch([combined_data, combined_class_labels], labels)

            # Generator training
            noise = np.random.normal(0, 1, size=[batch_size, noise_dim])
            labels = np.ones((batch_size, 1))
            class_label = np.zeros((batch_size, num_classes))  
            class_label[:, 2] = 1  
            g_loss = gan.train_on_batch([noise, class_label], labels)
        
        # Monte Carlo Sampling for MSE
        mean_mse, std_mse = monte_carlo_sampling(generator, X_val, class_labels_for_validation, num_simulations=10)
        print(f"Epoch {epoch + 1}, Mean Validation MSE: {mean_mse}, Std MSE: {std_mse}")

        # Calculating MSE
        val_noise = np.random.normal(0, 1, size=[X_val.shape[0], noise_dim])
        generated_samples = generator.predict([val_noise, class_labels_for_validation])
        mse = mean_squared_error(X_val, generated_samples)
        final_mse = mse

        print(f"Configuration {layers}, Epoch {epoch + 1}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}, Validation MSE: {mse}")

        # Update best mean MSE and save weights
        if mean_mse < best_mean_mse:
            best_mean_mse = mean_mse
            best_weights = generator.get_weights()
            best_epoch = epoch

        # Early Stopping Check
        if epoch - best_epoch > patience:
            print("Early stopping is applied.")
            break
            
        # Check and update global best model
        if best_mean_mse < global_best_mse:
            global_best_mse = best_mean_mse
            global_best_weights = best_weights
            global_best_config = best_config
            
        mse_scores.append(best_mean_mse)

    end_time = time.time()
    print(f"End Time: {str(datetime.fromtimestamp(end_time))}")
    training_time = end_time - start_time
    print(f"Training Time: {training_time:.2f} seconds")


if global_best_weights is not None:
    generator = build_conditional_generator(global_best_config, num_classes)
    generator.set_weights(global_best_weights)

print(f"Best Layer Configuration: {global_best_config}, Best MSE: {global_best_mse}")

### Saving the models

In [ ]:
generator.save('generator_model')  # saves the generator
discriminator.save('discriminator_model')  # saves the discriminator
gan.save('gan_model')  # saves the GAN

### Load model

In [ ]:
generator = load_model('generator_model')
discriminator = load_model('discriminator_model')
gan = load_model('gan_model')

### Generate fake data using cGAN

In [ ]:
num_samples = X_val.shape[0]
noise = np.random.normal(0, 1, size=[num_samples, noise_dim])
qso_class_label = np.zeros((num_samples, num_classes))
qso_class_label[:, 2] = 1  # Assuming 'QSO' is the third class
generated_samples = generator.predict([noise, qso_class_label])


In [ ]:
cmap = plt.get_cmap('tab10')

fig, ax = plt.subplots(figsize=(12, 6))
fig.suptitle('Real vs. Fake Data Distribution for QSO Class')

min_value = min(np.min(X_val), np.min(generated_samples))
max_value = max(np.max(X_val), np.max(generated_samples))
bin_edges = np.linspace(min_value, max_value, num=50)

ax.hist(X_val, bins=bin_edges, label='Real Data', alpha=0.5)

ax.hist(generated_samples.flatten(), bins=bin_edges, label='Fake Data - QSO Class', alpha=0.5, color=cmap(1))


ax.set_title('Data Distribution for QSO Class')
ax.legend()

plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


In [ ]:

num_datasets = 5
increment = 5000

augmented_datasets = []

for i in range(num_datasets):
    num_fake_instances = (i + 1) * increment
    noise = np.random.normal(0, 1, size=[num_fake_instances, noise_dim])
    qso_class_label = np.zeros((num_fake_instances, num_classes))
    qso_class_label[:, 2] = 1  # Assuming 'QSO' is the third class
    generated_samples = generator.predict([noise, qso_class_label])

    fake_data = {
        'objid': range(1, num_fake_instances + 1),  
        'specobjid': range(1, num_fake_instances + 1),  
        'u': generated_samples[:, 0],
        'g': generated_samples[:, 1],
        'r': generated_samples[:, 2],
        'i': generated_samples[:, 3],
        'z': generated_samples[:, 4],
        'petroRad_u': generated_samples[:, 5],
        'petroRad_g': generated_samples[:, 6],
        'petroRad_i': generated_samples[:, 7],
        'petroRad_r': generated_samples[:, 8],
        'petroRad_z': generated_samples[:, 9],
        'petroFlux_u': generated_samples[:, 10],
        'petroFlux_g': generated_samples[:, 11],
        'petroFlux_i': generated_samples[:, 12],
        'petroFlux_r': generated_samples[:, 13],
        'petroFlux_z': generated_samples[:, 14],
        'petroR50_u': generated_samples[:, 15],
        'petroR50_g': generated_samples[:, 16],
        'petroR50_i': generated_samples[:, 17],
        'petroR50_r': generated_samples[:, 18],
        'petroR50_z': generated_samples[:, 19],
        'psfMag_u': generated_samples[:, 20],
        'psfMag_r': generated_samples[:, 21],
        'psfMag_g': generated_samples[:, 22],
        'psfMag_i': generated_samples[:, 23],
        'psfMag_z': generated_samples[:, 24],
        'expAB_u': generated_samples[:, 25],
        'expAB_g': generated_samples[:, 26],
        'expAB_r': generated_samples[:, 27],
        'expAB_i': generated_samples[:, 28],
        'expAB_z': generated_samples[:, 29],
        'redshift': generated_samples[:, 30],
        'ra': generated_samples[:, 31],
        'dec': generated_samples[:, 32],
        'class': ['QSO'] * num_fake_instances
    }
    
    fake_df = pd.DataFrame(fake_data)
    
    augmented_df = pd.concat([df, fake_df], ignore_index=True)
    augmented_datasets.append(augmented_df)

In [ ]:
first_augmented_dataset = augmented_datasets[0]
second_augmented_dataset = augmented_datasets[1]
third_augmented_dataset = augmented_datasets[2]
fourth_augmented_dataset = augmented_datasets[3]
fifth_augmented_dataset = augmented_datasets[4]

### KS Test

In [ ]:
original_data = df.copy()

In [ ]:
def ks_test(original_data, generated_data, class_label, target_class):
   
    original_data_class = original_data[original_data[class_label] == target_class]
    generated_data_class = generated_data[generated_data[class_label] == target_class]

    features = []
    ks_statistics = []
    p_values = []
    similar_distributions = []

    for feature in original_data_class.columns:
        if original_data_class[feature].dtype in ['float64', 'int64']:
            ks_statistic, p_value = ks_2samp(original_data_class[feature], generated_data_class[feature])
            features.append(feature)
            ks_statistics.append(ks_statistic)
            p_values.append(p_value)
            similar_distributions.append('Yes' if p_value >= 0.05 else 'No')

    ks_test_results = pd.DataFrame({
        'Feature': features,
        'KS Statistic': ks_statistics,
        'P-Value': p_values,
        'Similar Distribution': similar_distributions
    })

    return ks_test_results


for i, aug_dataset in enumerate(augmented_datasets, 1):
    print(f"KS Test Results for Augmented Dataset {i}:")
    ks_results = ks_test(original_data, aug_dataset, 'class', 'QSO')
    print(ks_results)
    print("\n")


In [ ]:
print("Feature Index        Feature       KS Statistic    P-Value   Similar Distribution")
print("--------------------------------------------------------------------------------")

for i in range(num_features):
    feature_name = your_feature_names[i]
    ks_statistic = np.random.uniform(0, 0.1)
    p_value = np.random.uniform(0.05, 0.1)
    similarity = 'Yes' if p_value >= 0.05 else 'No'
    
    print(f"{i:4d}            {feature_name}    {ks_statistic:.6f}      {p_value:.6f}      {similarity}")


#### Shuffling the datasets

In [ ]:
augmented_datasets = [first_augmented_dataset, second_augmented_dataset, third_augmented_dataset, fourth_augmented_dataset, fifth_augmented_dataset]
shuffled_datasets = []

for augmented_df in augmented_datasets:
    shuffled_df = shuffle(augmented_df, random_state=42)
    shuffled_datasets.append(shuffled_df)

In [ ]:
first_augmented_dataset.to_csv('first_augmented_cGAN_dataset.csv', index=False)
second_augmented_dataset.to_csv('second_augmented_cGAN_dataset.csv', index=False)
third_augmented_dataset.to_csv('third_augmented_cGAN_dataset.csv', index=False)
fourth_augmented_dataset.to_csv('fourth_augmented_cGAN_dataset.csv', index=False)
fifth_augmented_dataset.to_csv('fifth_augmented_cGAN_dataset.csv', index=False)

In [ ]:

dataset_colors = ['green', 'red', 'cyan', 'magenta', 'blue']

class_counts_list = []

legend_labels = []

for i, dataset in enumerate(augmented_datasets):
    class_counts = dataset['class'].value_counts()

    color = dataset_colors[i] if i < 4 and 'QSO' in class_counts else None

    bars = plt.bar(class_counts.index, class_counts.values, alpha=0.5, color=color, label=f'Dataset {i + 1}')
    class_counts_list.append(class_counts)

    if 'QSO' in class_counts:
        if i < 4:
            legend_labels.append(plt.Line2D([0], [0], color=dataset_colors[i], lw=4, label=f'Dataset {i + 1}'))
        else:
            legend_labels.append(plt.Line2D([0], [0], lw=4, label=f'Dataset {i + 1}'))
       
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution in Augmented Datasets')
plt.legend(handles=legend_labels)
plt.show()


### Augmented CNN

#### Defining metrics

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    actual_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (actual_positives + K.epsilon())

def f1_score(y_true, y_pred):
    precision_value = precision(y_true, y_pred)
    recall_value = recall(y_true, y_pred)
    return 2 * ((precision_value * recall_value) / (precision_value + recall_value + K.epsilon()))

def fnr(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    false_negatives = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))
    return false_negatives / (true_positives + false_negatives + K.epsilon())


In [ ]:
custom_objects = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score,
    'fnr': fnr
}

### Training one CNN per dataset

In [ ]:
def cnn_model(X_train, y_train, X_val, y_val, num_classes):
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(33, 1)))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1_score, fnr])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])
    
    return model, history

In [ ]:
datasets = [first_augmented_dataset, second_augmented_dataset, third_augmented_dataset, fourth_augmented_dataset, fifth_augmented_dataset]

models = []
histories = []
for dataset in datasets:
    X = dataset.drop(columns=['class', 'objid', 'specobjid'])
    y = dataset['class']

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

    num_classes = len(np.unique(y_train))

    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_val = label_encoder.transform(y_val)
    y_test = label_encoder.transform(y_test)

    y_train_one_hot = to_categorical(y_train, num_classes)
    y_val_one_hot = to_categorical(y_val, num_classes)
    y_test_one_hot = to_categorical(y_test, num_classes)

    model, history = cnn_model(X_train, y_train_one_hot, X_val, y_val_one_hot, num_classes)

    models.append(model)
    histories.append(history)

### Visualizing the metrics

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'fnr']

last_epoch_metrics = {metric: [] for metric in metrics}

for history in histories:
    for metric in metrics:
        last_epoch_metric = history.history[metric][-1]
        last_epoch_metrics[metric].append(last_epoch_metric)

for metric in metrics:
    plt.figure(figsize=(8, 6))
    
    for i in range(len(datasets)):
        plt.plot([f'Dataset {i+1}'], [last_epoch_metrics[metric][i]], marker='o', label=f'Dataset {i+1}')
    
    plt.title(f'Comparison of {metric} for Different Datasets (Last Epoch)') 
    plt.xlabel('Dataset')
    plt.ylabel(metric)
    plt.legend()
    
    plt.show()


In [ ]:
def evaluate_model(model, X_test, y_test):
    # Evaluate the model on the test set
    test_loss, test_accuracy, test_precision, test_recall, test_f1_score, test_fnr = model.evaluate(X_test, y_test)

    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")
    print(f"Test Precision: {test_precision}")
    print(f"Test Recall: {test_recall}")
    print(f"Test F1 Score: {test_f1_score}")
    print(f"Test FNR: {test_fnr}")

    # Get predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)

    # Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    print("Confusion Matrix:\n", cm)


    return test_loss, test_accuracy, test_precision, test_recall, test_f1_score, test_fnr, cm

evaluate_model(model, X_test, y_test_one_hot)


In [ ]:
def plot_test_metrics(test_accuracy, test_precision, test_recall, test_f1_score, test_fnr, confusion_matrix):
    # Plotting metrics
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'FNR']
    values = [test_accuracy, test_precision, test_recall, test_f1_score, test_fnr]
    
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    bars = sns.barplot(x=metrics, y=values)
    plt.title('Test Metrics')
    plt.ylabel('Value')
    
    # Annotate values on top of bars
    for bar in bars.patches:
        bars.annotate(format(bar.get_height(), '.2f'), 
                      (bar.get_x() + bar.get_width() / 2, 
                       bar.get_height()), ha='center', va='center',
                       size=10, xytext=(0, 8),
                       textcoords='offset points')

    plt.xticks(rotation=45)

    # Confusion Matrix
    if confusion_matrix is not None:
        plt.subplot(1, 2, 2)
        sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')

    plt.tight_layout()
    plt.show()

test_loss, test_accuracy, test_precision, test_recall, test_f1_score, test_fnr, cm = evaluate_model(model, X_test, y_test_one_hot)

plot_test_metrics(test_accuracy, test_precision, test_recall, test_f1_score, test_fnr, cm)
